In [20]:
from huggingface_hub import notebook_login

notebook_login()


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!git clone https://github.com/Neilus03/GEMMA_ORCAMATH_FINETUNING.git

Cloning into 'GEMMA_ORCAMATH_FINETUNING'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 148 (delta 11), reused 17 (delta 6), pack-reused 122
Receiving objects: 100% (148/148), 60.55 MiB | 27.35 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [18]:
%cd GEMMA_ORCAMATH_FINETUNING
!pip install -r requirements.txt


/content/GEMMA_ORCAMATH_FINETUNING/GEMMA_ORCAMATH_FINETUNING
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-_7urvl47
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-_7urvl47
  Resolved https://github.com/huggingface/accelerate.git to commit 60bfdaa9349e3e1a373863a8c5f20baa89cfd000
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-z57k6zmq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-z57k6zmq
  Resolved https://github.com/huggingface/peft.git to commit 5471c9a1be61d798447a24660acada40892777ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingfac

Download Orcamath from the following link: [OrcaMath Dataset](https://drive.google.com/uc?id=1JHDUPlG5igZ1QZ0McNYmUKzixV9pJXIZ) and add it to the folder.

In [21]:
#You can either decompress it manually and store it in the data subfolder or run the following lines

!unzip "/content/orcamath_data.zip" -d "/content/GEMMA_ORCAMATH_FINETUNING/data"


Archive:  /content/orcamath_data.zip
  inflating: /content/GEMMA_ORCAMATH_FINETUNING/data/orcamath_data.json  


To finetune the model on the OrcaMath dataset run the command below

In [22]:


!chmod +x run.sh
!./run.sh

dir: /content/GEMMA_ORCAMATH_FINETUNING/GEMMA_ORCAMATH_FINETUNING
2024-03-12 10:09:40.164884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 10:09:40.164944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 10:09:40.167089: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-12 10:09:42.009705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(load_in_4bit=True, load_in_8bit=False, use_lora=True, lora_r=8, lora_alpha=16, lora_target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj']) True
model: google/gemma

To do inference with the model just run the following cell and compare results

In [26]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import safetensors.torch as torchst
import torch
from peft import PeftModel
import torch
import os
from huggingface_hub import HfFolder

# Check if the token is already saved
token_path = HfFolder.path_token
if not os.path.exists(token_path):
    # If the token is not saved, ask for it and save it
    access_token = input("Add your gemma HF access token: ")
    HfFolder.save_token(access_token)  # Save the token for future use
else:
    print("HF access token is already saved.")


# Assuming 'checkpoint_path' is the path to your 'checkpoint-50010' directory
checkpoint_path = "./weights/e10_gemma_2b_qvko_r8_a16_lr5e-5_bs12/checkpoint-50010"

# Load the model from the Hugging Face Hub (without the adapter head)
pretrained_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b", # Load the model from the Hugging Face Hub (without the adapter head)
    device_map="auto", # Use the default device (GPU if available, CPU otherwise)
    trust_remote_code=True,
)

# Load the tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)


# Load the adapter head from the local directory
model = PeftModel.from_pretrained(pretrained_model, checkpoint_path)

# Merge the adapter head into the model and unload it. This will make the model ready for inference.
finetuned_model = model.merge_and_unload().to("cuda")

# Example of inference
input_text = "explain exponentiation to a child in demographic terms"

# Tokenize the input text and send it to the GPU
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate the output of the pretrained model for the input text
outputs = pretrained_model.generate(**input_ids, max_length=400, num_return_sequences=1)
print("Pretrained model output:")
print(tokenizer.decode(outputs[0]))
print("--------------------------------------\n")
# Generate the output of the finetuned model for the input text
print("Finetuned model output:")
outputs = finetuned_model.generate(**input_ids, max_length=400, num_return_sequences=1)
print(tokenizer.decode(outputs[0]))


HF access token is already saved.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.